# Bowser Widget Example Usage

This notebook demonstrates how to use the Bowser ipywidget for interactive data exploration in Jupyter notebooks.

## Method 1: Automatic Server Management

The simplest approach - the widget will automatically start a Bowser server if one isn't already running:

In [1]:
import os
%load_ext autoreload
%autoreload 2
os.chdir("/Users/staniewi/repos/opera-utils/data")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import bowser._widget

# This will automatically start a server if needed
# and create a widget for the first available dataset
widget = bowser._widget.make_bowser_widget(
    # stack_file="data/new-orleans-F44055-2048.zarr",  # or your zarr/netcdf file
    rasters_file="bowser_rasters_reso.json",  # or your JSON config file for COG mode
)

widget

INFO:bowser._widget:Starting Bowser server...
INFO:bowser._server:Starting Bowser server on http://127.0.0.1:8000
INFO:bowser:Loading RasterGroups from bowser_rasters_reso.json (COG)
INFO:bowser:Found 22 RasterGroup configs: {list(raster_groups.keys())}
INFO:bowser:Configured COG endpoints at /cog/*
INFO:bowser:Configured MD endpoints at /md/*
INFO:bowser:Bowser started in COG mode


Data loading complete in 2.2 sec. Mode: cog
Setup complete: time to load datasets: 2.4 sec.


INFO:bowser._widget:Updated reference values for 150m: Displacement


INFO:bowser._widget:Updated reference values for 150m: Displacement


In [4]:
from bowser._server import BowserServer
from bowser._widget import make_bowser_widget

# Start the server manually
server = BowserServer(
    stack_file="data/west-texas-tubbs-uplift.zarr",
    port=8001,  # Use a specific port
    ignore_sidecar_files=True,
)
server.start()

print(f"Server running at: {server.url}")

INFO:bowser._server:Starting Bowser server on http://127.0.0.1:8001


Server running at: http://127.0.0.1:8001


In [5]:
# Create widget pointing to the running server
widget = make_bowser_widget(
    dataset_name="displacement",  # Specify which dataset to view
    bowser_url=server.url
)

widget

ValueError: Dataset 'displacement' not found.

In [ ]:
# Stop the server when done
server.stop()

In [ ]:
# First, create a JSON configuration file for your rasters
import json

# Example configuration for COG files
raster_config = [
    {
        "name": "displacement",
        "file_list": [
            "data/geotiffs/displacement_20160708_20160801.tif",
            "data/geotiffs/displacement_20160708_20160825.tif",
            "data/geotiffs/displacement_20160708_20160918.tif"
        ],
        "mask_file_list": [
            "data/geotiffs/recommended_mask_20160708_20160801.tif",
            "data/geotiffs/recommended_mask_20160708_20160825.tif",
            "data/geotiffs/recommended_mask_20160708_20160918.tif"
        ],
        "mask_min_value": 0.1,
        "uses_spatial_ref": True,
        "algorithm": "shift"
    }
]

# Save configuration
with open("bowser_rasters.json", "w") as f:
    json.dump(raster_config, f, indent=2)

In [ ]:
# Use COG mode
with running(
    rasters_file="bowser_rasters.json",
    ignore_sidecar_files=True
) as server:
    widget = make_bowser_widget(
        dataset_name="displacement",
        bowser_url=server.url
    )
    display(widget)

## Additional Features

The widget provides:

- **Interactive map**: Pan, zoom, and explore your geospatial data
- **Time series navigation**: Use the slider to move through time steps
- **Automatic server management**: No need to manually configure Titiler
- **Support for both data modes**: Works with both zarr/netcdf (MD mode) and COG files
- **Customizable**: Control zoom levels, loading indicators, and more

### Widget Parameters

- `dataset_name`: Which dataset to display (if None, uses first available)
- `bowser_url`: URL of Bowser server (if server already running)
- `show_loading`: Whether to show loading indicators on tiles
- `**server_kwargs`: Additional parameters for BowserServer (port, ignore_sidecar_files, etc.)